### 拳擊辨識系統 Boxing Identification System (BIS)

#### Import Resources

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

#### Data

In [ ]:
DATASET_DIR = "dataset"   # 資料夾結構
CLASSES = ["front", "side", "back"]
SEQUENCE_LENGTH = 30      # 每個動作序列包含 30 個 frames (可依影片fps調整)
KEYPOINT_DIM = 2          # MoveNet 2D keypoints: x, y
NUM_KEYPOINTS = 17        # MoveNet 預設17個關鍵點
NUM_CLASSES = len(CLASSES)

#### 讀取 Move Net Features

In [ ]:
def load_movenet_data():
    X = []
    Y = []

    for label_idx, label_name in enumerate(CLASSES):
        class_dir = os.path.join(DATASET_DIR, label_name)
        if not os.path.exists(class_dir):
            continue

        for file in os.listdir(class_dir):
            if file.endswith(".npy"):
                path = os.path.join(class_dir, file)
                sequence = np.load(path)  # shape (frames, 17, 2)

                # 若影片長度超過 SEQUENCE_LENGTH，就取前面一段
                if sequence.shape[0] > SEQUENCE_LENGTH:
                    sequence = sequence[:SEQUENCE_LENGTH]
                # 若不足，則補0
                elif sequence.shape[0] < SEQUENCE_LENGTH:
                    pad_len = SEQUENCE_LENGTH - sequence.shape[0]
                    pad = np.zeros((pad_len, NUM_KEYPOINTS, KEYPOINT_DIM))
                    sequence = np.concatenate([sequence, pad], axis=0)

                X.append(sequence)
                Y.append(label_idx)

    X = np.array(X)  # (N, 30, 17, 2)
    Y = np.array(Y)  # (N,)
    print(f"✅ Loaded data: {X.shape[0]} samples, shape per sample {X.shape[1:]}")

    return X, Y

#### Data Split and Normalization

In [ ]:
def prepare_dataset(X, Y):
    # 正規化: 把 keypoint 座標壓在 [0,1]
    X = X / np.max(X)

    # 分割訓練 / 驗證 / 測試
    X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.3, stratify=Y, random_state=42)
    X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, stratify=Y_temp, random_state=42)

    # one-hot encoding
    Y_train = to_categorical(Y_train, NUM_CLASSES)
    Y_val = to_categorical(Y_val, NUM_CLASSES)
    Y_test = to_categorical(Y_test, NUM_CLASSES)

    print(f"🧩 Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
    return X_train, X_val, X_test, Y_train, Y_val, Y_test

#### CNN - LSTM Model

In [ ]:
def build_cnn_lstm_model():
    model = models.Sequential([
        layers.TimeDistributed(layers.Conv2D(32, (3, 2), activation='relu', padding='same'),
                               input_shape=(SEQUENCE_LENGTH, NUM_KEYPOINTS, KEYPOINT_DIM, 1)),
        layers.TimeDistributed(layers.MaxPooling2D((2, 1))),
        layers.TimeDistributed(layers.Conv2D(64, (3, 2), activation='relu', padding='same')),
        layers.TimeDistributed(layers.Flatten()),
        layers.LSTM(128, return_sequences=False),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.4),
        layers.Dense(NUM_CLASSES, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

#### Main Code

In [ ]:
if __name__ == "__main__":
    # 載入資料
    X, Y = load_movenet_data()

    # 整理 dataset
    X_train, X_val, X_test, Y_train, Y_val, Y_test = prepare_dataset(X, Y)

    # 增加 channel 維度 (for Conv2D)
    X_train = X_train[..., np.newaxis]
    X_val = X_val[..., np.newaxis]
    X_test = X_test[..., np.newaxis]

    # 建立模型
    model = build_cnn_lstm_model()

    # 訓練
    history = model.fit(
        X_train, Y_train,
        epochs=30,
        batch_size=16,
        validation_data=(X_val, Y_val)
    )

    # 測試
    test_loss, test_acc = model.evaluate(X_test, Y_test)
    print(f"🎯 Test Accuracy: {test_acc:.4f}")